In [1]:
# ! pip install pandarallel
# from pandarallel import pandarallel
import json
import re
import pandas as pd
import numpy as np
import itertools

In [2]:
df = pd.read_csv('EX2_PPSWR.csv')

# remove blank spaces from front and back of column names
df.columns = df.columns.str.strip()
df.columns

C:\Users\gaurav\AppData\Local\Temp\ipykernel_8464\819888444.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('EX2_PPSWR.csv')


Index(['SEC', 'ST', 'AST', 'DC', 'NSSR', 'STRM', 'SSTRM', 'FSU', 'SSS', 'SSU',
       'CAPZS', 'SMLZS', 'DSI', 'NFSU', 'NSS', 'MULT', 'CAPH', 'SMLH', 'SRL',
       'HHSIZE', 'RELIGION', 'SG', 'MPCE_ALL', 'MPCE_ST', 'REL', 'SEX', 'AGE',
       'MARST', 'GEDU', 'TEDU'],
      dtype='object')

In [3]:
## Data is the dictionary
data = {}

## Keys
key = ''

## values
values = []

## Open
with open('Example2-PPSWR.RSE', "r") as f:
    print(type(f))
    for line in f:
        print(line)
        line = line.strip()
        if line.startswith('#'):
            if key:
                key = key.split("'")[0]
                key = key.split("\t")[0]
                data[key] = [v for v in values if v and not v.startswith("'")]
            key = line.split("'")[0]
            key = key.split("\t")[0]
            values = []
        elif line.startswith("'"):
            pass
        else:
            line = line.split("'")[0]
            values.append(line.strip())

# add last group
if key:
    data[key] = [v for v in values if v and not v.startswith("'")]

print(json.dumps(data, indent=4))

<class '_io.TextIOWrapper'>
'Example2.RSE â€“ file will be as follows:

#SAMPLING PPSWR-SSS	'PPSWR-SSS Key word will be used for this scheme

#GROUP 2			'Number of Group variables used

SEC   C sec 	'Sector Rural-1, Urban-2

AGE_GR C NEWVAR 	'Age Groups



#FILTER 0		'Number of conditions used to generate the Work-file



#VARIABLE 4  	'Number of Variables selected

YP	N  NEWVAR	'Variable for Population

YM	N  NEWVAR	'Variable for Male

YF	N  NEWVAR	'Variable for Female

YH	N  NEWVAR	'Variable for Higher Secondary educated persons



#RENAME  5    			'Number of variables to be renamed

STRMID C  sec+st+strm+sstrm	'Combination of fields to get unique Stratum ID

FSU    C  fsu			' FSU code

SSS    C  sss			'Second Stage Stratum (SSS) Code

USU    C  ssu			'Ultimate stage unit, here Households

NFSU   C  nfsu			'No. of FSU surveyed in STRMID

NSS    N  nss			'No. of FSU surveyed in STRMID X SSS

MULT   N  mult			'Multiplier X 100



#TRANSFORM 5  	' Number of Variable transformed

AGE_GR=

In [4]:
def SAMPLING_(data, df):
    for key, value in data.items():
        ## To read the sampling method used
        if str('SAMPLING').lower() in str(key).lower():
            if str('SUBSAMPLE2').lower() in str(key).lower():
                return str('SUBSAMPLE2')

            elif str('SRSWR').lower() in str(key).lower():
                pass
            elif str('SRSWOR').lower() in str(key).lower():
                pass

SAMPLING_(data, df)

In [5]:
### ---------------------------------------------------------------------------------------------------------
def GROUP_(data, df):
    for key, value in data.items():
        if str('GROUP').lower() in str(key).lower():
            value = [s.split()[0] for s in value]
            for name in value:
                if name in df.columns:
                    pass
                elif name not in df.columns:
                    df[name] = 0

            return value


    for key, value in data.items():
        print(key, value)
        ### To read the group columns
        if str('GROUP').lower() in str(key).lower():
            # Define regular expression to match three words separated by whitespace
            regex = r'\w+\s+\w+\s+\w+'

            for i in range(len(value)):
                # Find the first match in the string
                match = re.search(regex, value[i])

                # Print the match
                lst = match.group().split()

                print(lst[0], lst[1])

                if lst[0] in df.columns:
                    if lst[1] == 'C':
                        df[lst[0]] = df[lst[0]].astype(str)
                elif lst[0] not in df.columns:
                    # if lst[1] == 'C':
                    #     df[lst[0]] = ''
                    pass

        else:
            pass

### ---------------------------------------------------------------------------------------------------------
def FILTER_(data, df):
    lst_queries = []
    for key, value in data.items():

        if str('FILTER').lower() in str(key).lower():
            # Define regular expression to match three words separated by whitespace
            regex = r'\([^()]+\)|\w+[<>!=]+[0-9]+(?:\s*(?:&&|\|\|)\s*(?:\([^()]+\)|\w+[<>!=]+[0-9]+))*'

            for i in range(len(value)):
                # Find the first match in the string
                match = re.search(regex, value[i])

                # Print the match
                lst_queries.append(match.group())

        else:
            pass

    filter_query = ' & '.join(['({})'.format(item) for item in lst_queries])

    try:
        df = df.query(filter_query)
        return df
    except:
        return df


### ---------------------------------------------------------------------------------------------------------
def FILE_(data):
    for key, value in data.items():
        ### To read the file used
        if str('FILE').lower() in str(key).lower():
            if key.strip().endswith('.xlsx') or key.strip().endswith('.csv'):
                match = re.search(r'[A-Za-z0-9]+\.xlsx|[A-Za-z0-9]+\.csv', key)
                if match:
                    file_name = match.group(0)
                    return file_name
                else:
                    file_name = None
                    return file_name
                print(file_name)

            # return file_name

### ---------------------------------------------------------------------------------------------------------
def RENAME_(data, df):
    # create an empty dictionary to store the column names
    column_dict = {}

    for key, value in data.items():
        ### To read the file used
        if str('RENAME').lower() in str(key).lower():
            # value = [elem for elem in value[0].split(' ') if elem.strip()]
            # return value

            for col in value:
                col_split = col.split()

                # print(col_split[0])

                # extract the last element as the column name
                col_name = col_split[-1]

                new_col_name = col_split[0]

                ### if there is a "+" sign in the last element, concatenate the columns
                if "+" in col_name:
                    col_concat = col_name.split("+")
                    col_concat = [i.upper() for i in col_concat]

                    # concatenate the columns
                    # df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if i < 10 else str(i) for i in x]), axis=1)
                    # df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if i < 10 else str(i) for i in x]), axis=1)
                    df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if int(i) < 10 else str(i) for i in x]), axis=1)


                else:
                    col_name = col_name.upper()

### ---------------------------------------------------------------------------------------------------------
def NEWVARIABLE_(data, df):
    for key, value in data.items():
        ### To read the file used
        if str('VARIABLE').lower() in str(key).lower():
            value = [s.split()[0] for s in value]

            for name in value:
                df[name] = 1

### ---------------------------------------------------------------------------------------------------------
def TRANSFORM_(data):
    global condition

    # Initialize empty arrays to store the values
    Arr1 = []
    Arr2 = []
    Arr3 = []

    for key, value in data.items():
        ### To read the file used
        if str('TRANSFORM').lower() in str(key).lower():
            lines = '\n'.join(value)
            lines = lines.split("\n")

            # Loop through the lines and extract the values
            for line in lines:
                line_values = line.split('=')  # split the line into values
                print(line_values)
                var_name = line_values[0].strip()  # extract the variable name

                var_value = line_values[1].strip().split()[0]  # extract the variable value


                # Regular expression pattern to extract the condition
                pattern = r'^\w+=\([^\)]*\)(?:\s+(.*?)(?:\n|$))?$'

                # Split the string into lines and extract the condition from each line
                conditions = []

                for line in line.split('\n'):
                    match = re.match(pattern, line)

                    if match:
                        condition = match.group(1)

                        if condition is not None:
                            conditions.append(condition.strip())
                        else:
                            conditions.append(None)

                # Append the values to the arrays
                Arr1.append(var_name)
                Arr2.append(var_value)

                Arr3.append(condition)

    return Arr1, Arr2, Arr3

### ---------------------------------------------------------------------------------------------------------
def EST_RSE_(data):
    for key, value in data.items():
        pass

df = FILTER_(data, df)
GROUP_(data, df)
TRANSFORM_(data)
RENAME_(data, df)
NEWVARIABLE_(data, df)
FILE_(data)



['AGE_GR', '("01") AGE<5']
['AGE_GR', '("02") AGE>', '5 && AGE<10']
['AGE_GR', '("03") AGE>', '10 && AGE<20']
['AGE_GR', '("04") AGE>', '20 && AGE<60']
['AGE_GR', '("05") AGE>', '60']
['YP', '(1)']
['YM', '(1)        SEX<>2']
['YM', '(0)\t\tOtherwise']
['YF', '(1)        SEX', '2']
['YF', '(0)\t\tOtherwise']
['YH', '(1)        GEDU>', '"11"']
['YH', '(0)\t\tOtherwise']


In [6]:
def parse_query_string(query_string):
    # Replace curly quotes with straight quotes
    query_string = query_string.replace("”", "\"")

    if 'in' in query_string:
        if ('NOT in' or 'not in') in query_string:
            query_string = re.sub(r'(\b\w+\b)\s+NOT in\s*\(([^)]+)\)', r'~\1.isin([\2])', query_string)
            query_string = re.sub(r'(\b\w+\b)\s+in\s*\(([^)]+)\)', r'\1.isin([\2])', query_string)
        elif 'in' in query_string:
            # Replace 'in' operators with 'isin' and add list brackets
            query_string = re.sub(r'(\b\w+\b)\s+in\s*\(([^)]+)\)', r'\1.isin([\2])', query_string)
        else:
            pass

    else:
        query_string = re.sub(r'''(\b\w+\b)(\s*[<>=!]+\s*)('[^']*'|"[^"]*")|(\b\w+\b)(\s*[<>=!]+\s*)("[^]*'|"[^"]*")''', r'\1\2\3', query_string)
        query_string = re.sub(r'''(\b\w+\b)(\s*[<>=!]+\s*)(\d+)''', r'\1\2\3', query_string)

    # Replace double quotes with single quotes
    query_string = query_string.replace("\"", "'")

    # Replace 'AND' with '&', 'OR' with '|', and add parentheses
    query_string = query_string.replace(" AND ", " & ")\
                        .replace(" OR ", " | ").\
                            replace("&&", '&').\
                                replace('||', '|').\
                                    replace('<>', '!=').\
                                        replace('NOT', '~')

    # Replace column names with df[column] syntax
    query_string = re.sub(r'(\b\w+\b)=', r'\1==', query_string)


    # # Define a function to replace variable names with df["<variable_name>"]
    # def replace_var(match):
    #     return f'df["{match.group(0)}"]'

    # # Use regex to replace variable names with df["<variable_name>"]
    # pattern = re.compile(r'''\b\w+\b(?=\s*[<>=!]=*\s*[\'"\d])''')
    # new_string = pattern.sub(replace_var, query_string)

    query_string = re.sub(r"'(\d+)'|\"(\d+)\"", lambda match: str(int(match.group(1) or match.group(2))), query_string)
    query_string = re.sub(r'\[(\d+,\s*)+\d+\]', lambda match: '[' + match.group(0).replace("\'", "'") + ']', query_string)

    return query_string


final_parsed_query = []

for i in list(TRANSFORM_(data))[2]:
    # print(i)
    if i:
        parsed_query = parse_query_string(i)
        final_parsed_query.append(parsed_query)
    else:
        final_parsed_query.append(i)

### Calling the final_parsed_query function
### ---------------------------------------
final_parsed_query

['AGE_GR', '("01") AGE<5']
['AGE_GR', '("02") AGE>', '5 && AGE<10']
['AGE_GR', '("03") AGE>', '10 && AGE<20']
['AGE_GR', '("04") AGE>', '20 && AGE<60']
['AGE_GR', '("05") AGE>', '60']
['YP', '(1)']
['YM', '(1)        SEX<>2']
['YM', '(0)\t\tOtherwise']
['YF', '(1)        SEX', '2']
['YF', '(0)\t\tOtherwise']
['YH', '(1)        GEDU>', '"11"']
['YH', '(0)\t\tOtherwise']


['AGE<5',
 'AGE>=5 & AGE<10',
 'AGE>=10 & AGE<20',
 'AGE>=20 & AGE<60',
 'AGE>=60',
 None,
 'SEX!=2',
 'Otherwise',
 'SEX==2',
 'Otherwise',
 'GEDU>=11',
 'Otherwise']

In [7]:
Arr1 = list(TRANSFORM_(data))[0]
Arr2 = list(TRANSFORM_(data))[1]

# print(Arr2)
Arr2 = [i.strip('()\'\"') for i in Arr2]
Arr3 = final_parsed_query

Arr2 = [float(elem) if elem.replace('.', '', 1).isdigit() else elem for elem in Arr2]
print(Arr2)

# print(Arr1)
# print(Arr2)
# print(Arr3)

['AGE_GR', '("01") AGE<5']
['AGE_GR', '("02") AGE>', '5 && AGE<10']
['AGE_GR', '("03") AGE>', '10 && AGE<20']
['AGE_GR', '("04") AGE>', '20 && AGE<60']
['AGE_GR', '("05") AGE>', '60']
['YP', '(1)']
['YM', '(1)        SEX<>2']
['YM', '(0)\t\tOtherwise']
['YF', '(1)        SEX', '2']
['YF', '(0)\t\tOtherwise']
['YH', '(1)        GEDU>', '"11"']
['YH', '(0)\t\tOtherwise']
['AGE_GR', '("01") AGE<5']
['AGE_GR', '("02") AGE>', '5 && AGE<10']
['AGE_GR', '("03") AGE>', '10 && AGE<20']
['AGE_GR', '("04") AGE>', '20 && AGE<60']
['AGE_GR', '("05") AGE>', '60']
['YP', '(1)']
['YM', '(1)        SEX<>2']
['YM', '(0)\t\tOtherwise']
['YF', '(1)        SEX', '2']
['YF', '(0)\t\tOtherwise']
['YH', '(1)        GEDU>', '"11"']
['YH', '(0)\t\tOtherwise']
[1.0, 2.0, 3.0, 4.0, 5.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0]


In [8]:

# Check for name mismatches and create new columns
mismatched_names = set(Arr1) - set(df.columns)

new_columns = {name: [0] * df.shape[0] for name in mismatched_names}
df = df.assign(**new_columns)

mismatched_names

set()

In [9]:
Arr3 = [i.replace("(['", "([").replace("'])", "])") if i is not None else 'True' if Arr3[j] == 'Otherwise' else True for j, i in enumerate(Arr3)]
Arr3

['AGE<5',
 'AGE>=5 & AGE<10',
 'AGE>=10 & AGE<20',
 'AGE>=20 & AGE<60',
 'AGE>=60',
 True,
 'SEX!=2',
 'Otherwise',
 'SEX==2',
 'Otherwise',
 'GEDU>=11',
 'Otherwise']

In [10]:
new_arr3 = []
for i, value in enumerate(Arr3):
    if value == 'Otherwise':
        other_values = set(['~(' + Arr3[j] + ')' for j in range(len(Arr3)) if Arr1[j] == Arr1[i] and Arr3[j] != 'Otherwise'])
        new_value = ' & '.join(other_values)
        print(new_value)
        new_arr3.append(new_value)
    else:
        new_arr3.append(value)
        print(value)

# new_arr3

AGE<5
AGE>=5 & AGE<10
AGE>=10 & AGE<20
AGE>=20 & AGE<60
AGE>=60
True
SEX!=2
~(SEX!=2)
SEX==2
~(SEX==2)
GEDU>=11
~(GEDU>=11)


In [11]:
Arr3 = new_arr3

In [12]:
for i in range(len(Arr2)):
    if type(Arr3[i]) == str:
        print(Arr3[i])

AGE<5
AGE>=5 & AGE<10
AGE>=10 & AGE<20
AGE>=20 & AGE<60
AGE>=60
SEX!=2
~(SEX!=2)
SEX==2
~(SEX==2)
GEDU>=11
~(GEDU>=11)


In [13]:
# Apply the queries in Arr3 to filter the rows of the dataframe
for i, query_str in enumerate(Arr3):

    if query_str is True:
        if type(Arr2[i]) == str:
            if '+' in Arr2[i]:
                df[Arr1[i]] = df[Arr2[i].split('+')].apply(lambda x: x.sum(), axis=1)
                print('*** First if', Arr2[i], Arr1[i], Arr3[i], i)

            else:
                df[Arr1[i]] = df[[Arr2[i]]].apply(lambda x: x, axis=1)
                print('*** First else', Arr2[i], Arr1[i], Arr3[i], i)
        # if type(Arr3[i]):
        #     print('^^^ Second if', Arr2[i], Arr1[i], Arr3[i], i)
        #     df.loc[df_query.index, Arr1[i]] = Arr2[i]

    else:
        # try:
        # query_str = re.sub(r'0?\s?(\d+)', r'\1', query_str)
        print(query_str)
        df_query = df.query(query_str)
        # except:
        #     pass

        if type(Arr2[i]) == float:
            df.loc[df_query.index, Arr1[i]] = Arr2[i]
            # print('### Third if', Arr2[i], Arr1[i], Arr3[i])
        else:
            df.loc[df_query.index, Arr1[i]] = Arr2[i]
            # print('@@@ First else', Arr2[i], Arr1[i], Arr3[i])


        if type(Arr2[i]) == str:
            if Arr2[i] in list(df.columns):
                df[Arr1[i]] = df[[Arr2[i]]].apply(lambda x: x, axis=1)
                # print('$$$ Fourth if', Arr2[i], Arr1[i], Arr3[i])

# print(df)

AGE<5
AGE>=5 & AGE<10
AGE>=10 & AGE<20
AGE>=20 & AGE<60
AGE>=60
SEX!=2
~(SEX!=2)
SEX==2
~(SEX==2)
GEDU>=11
~(GEDU>=11)


In [14]:
# df = pd.read_csv('WF_PPSWR.csv')
# df.head()

In [15]:
# Remove leading and trailing spaces from column names
# df.columns = df.columns.str.strip()

In [16]:
# new_col_names = {'sec':'SEC', 'age':'AGE', 'age_gr':'AGE_GR', 'strmid':'STRMID', 
#                  'fsu':'FSU', 'sss':'SSS', 'capzs':'CAPZS', 'smlzs':'SMLZS', 'nfsu':'NFSU', 
#                  'nss':'NSS', 'mult':'MULT', 'sex':'SEX', 'yp':'YP', 'ym':'YM', 'yf':'YF', 
#                  'gedu':'GEDU', 'yh':'YH', 'cmult':'CMULT'}

# df.rename(columns=new_col_names, inplace=True)

In [17]:
df['MULT'] = pd.to_numeric(df['MULT'])

# Calculate the SS_MULT and CMULT fields
df['SS_MULT'] = df['MULT'] / 100
df['CMULT'] = df['MULT'] / 100


In [18]:
df.columns

Index(['SEC', 'ST', 'AST', 'DC', 'NSSR', 'STRM', 'SSTRM', 'FSU', 'SSS', 'SSU',
       'CAPZS', 'SMLZS', 'DSI', 'NFSU', 'NSS', 'MULT', 'CAPH', 'SMLH', 'SRL',
       'HHSIZE', 'RELIGION', 'SG', 'MPCE_ALL', 'MPCE_ST', 'REL', 'SEX', 'AGE',
       'MARST', 'GEDU', 'TEDU', 'AGE_GR', 'STRMID', 'YP', 'YM', 'YF', 'YH',
       'SS_MULT', 'CMULT'],
      dtype='object')

In [19]:

df = df[['SEC', 'AGE', 'AGE_GR', 'STRMID', 'FSU', 'SSS', 'SSU', 'CAPZS', 'SMLZS', 'NFSU', 'NSS', 'MULT', 'SEX', 'YP', 'YM', 'YF', 'GEDU', 'YH', 'CMULT']]
df.to_csv('WorkFile_PPSWR_Method_Example1.csv')

In [20]:
# df = pd.read_excel('new_wf_ppswr_122532.xlsx')
# df = df.rename(columns=lambda x: x.strip())

In [21]:
bin(8)

'0b1000'

In [22]:
def generate_binary_combinations(n):
    binary_combinations = []
    for i in range(2 ** n):
        binary_combination = bin(i)[2:]
        binary_combination = '0' * (n - len(binary_combination)) + binary_combination
        binary_combinations.append(binary_combination)
    return binary_combinations

# limit = int(input("Enter - "))
limit = 2
binary_of_no = generate_binary_combinations(limit)
print(binary_of_no)

['00', '01', '10', '11']


In [23]:
def generate_variable_mapping(GROUP, binary_combinations):
    mapping = {}
    
    for comb in binary_combinations:
        variable_values = []
        for i, bit in enumerate(comb):
            if bit == '0':
                variable_values.append('')
            else:
                variable_values.append(GROUP[i])
        mapping[comb] = variable_values
    
    return mapping

GROUP = ['SEC', 'ST_GR']
n = len(GROUP)
binary_combinations = generate_binary_combinations(n)
variable_mapping = generate_variable_mapping(GROUP, binary_combinations)

print("Customized Mapping:")
for comb, values in variable_mapping.items():
    print(f"Binary Combination '{comb}': {values}")

Customized Mapping:
Binary Combination '00': ['', '']
Binary Combination '01': ['', 'ST_GR']
Binary Combination '10': ['SEC', '']
Binary Combination '11': ['SEC', 'ST_GR']


In [24]:
# # Step 5: Calculate variance of variables Y, X & R at STRMID level
# w6 = w5.groupby(['SEC', 'AGE_GR', 'STRMID', 'NS', 'SSS', 'nd_fsu']).agg(
#     myfym=('NS', lambda x: np.sum(((x * (w5['yffsu'] - w5['r1hat'] * w5['ymfsu'])) - (w5['yfstr'] - w5['r1hat'] * w5['ymstr'])) ** 2)),
#     vyh=('NS', lambda x: np.sum((x * w5['yhfsu'] - w5['yhstr']) ** 2)),
#     myhyp=('NS', lambda x: np.sum(((x * (w5['yhfsu'] - w5['r2hat'] * w5['ypfsu'])) - (w5['yhstr'] - w5['r2hat'] * w5['ypstr'])) ** 2))
# ).reset_index()

# w6

In [25]:
df.columns

Index(['SEC', 'AGE', 'AGE_GR', 'STRMID', 'FSU', 'SSS', 'SSU', 'CAPZS', 'SMLZS',
       'NFSU', 'NSS', 'MULT', 'SEX', 'YP', 'YM', 'YF', 'GEDU', 'YH', 'CMULT'],
      dtype='object')

In [26]:
import pandas as pd
import numpy as np

# Assuming you have a DataFrame 'wf_ppswr' with the necessary columns

# Step 1: Generate Group level estimates Yhat & Xhat
df['YP'] = pd.to_numeric(df['YP']) * df['CMULT']
df['YM'] = pd.to_numeric(df['YM']) * df['CMULT']
df['YF'] = pd.to_numeric(df['YF']) * df['CMULT']
df['YH'] = pd.to_numeric(df['YH']) * df['CMULT']

w1 = df.groupby(['SEC', 'AGE_GR']).agg(
    no_sam=('YP', 'size'),
    yphat=('YP', 'sum'),
    ymhat=('YM', 'sum'),
    yfhat=('YF', 'sum'),
    yhhat=('YH', 'sum')
).reset_index()

w1

w1.to_excel('W1_TEST.xlsx')

In [27]:
df

,SEC,AGE,AGE_GR,STRMID,FSU,SSS,SSU,CAPZS,SMLZS,NFSU,NSS,MULT,SEX,YP,YM,YF,GEDU,YH,CMULT
0,1,45.0,4,01010511,10001,1,1,941,1,2,2.0,47050,1,470.5,470.5,0.0,10.0,0.0,470.5
1,1,40.0,4,01010511,10001,1,1,941,1,2,2.0,47050,2,470.5,0.0,470.5,1.0,0.0,470.5
2,1,18.0,3,01010511,10001,1,1,941,1,2,2.0,47050,2,470.5,0.0,470.5,1.0,0.0,470.5
3,1,15.0,3,01010511,10001,1,1,941,1,2,2.0,47050,2,470.5,0.0,470.5,8.0,0.0,470.5
4,1,70.0,5,01010511,10001,3,1,941,1,2,2.0,47050,1,470.5,470.5,0.0,6.0,0.0,470.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576564,2,10.0,3,02363101,27122,7,1,712,1,2,2.0,2937000,2,29370.0,0.0,29370.0,7.0,0.0,29370.0
576565,2,73.0,5,02363101,27122,7,2,712,1,2,2.0,2937000,1,29370.0,29370.0,0.0,10.0,0.0,29370.0
576566,2,65.0,5,02363101,27122,7,2,712,1,2,2.0,2937000,2,29370.0,0.0,29370.0,1.0,0.0,29370.0
576567,2,31.0,4,02363101,27122,7,2,712,1,2,2.0,2937000,3,29370.0,29370.0,0.0,16.0,29370.0,29370.0


In [28]:
# import pandas as pd
# import numpy as np

# # Creating a sample dataframe with 10 columns
# data = {'A': ['foo', 'bar', 'foo', 'bar', 'foo', 'bar', 'foo', 'foo'],
#         'B': ['one', 'one', 'three', 'three', 'two', 'three', 'one', 'three'],
#         'C': [1, 2, 3, 4, 5, 3, 1, 3],
# }
# df1 = pd.DataFrame(data)

# df1 = df1.astype(str)

# df1['new_col'] = df1.groupby([f'col{i}' for i in range(6)])['col7'].transform('nunique')

# df1

In [29]:


df['FSU'] = df['FSU'].astype(str)
df['SSS'] = df['SSS'].astype(str)

df['UN_FSU'] = df['FSU'] + df['SSS']

w2 = df.groupby(['SEC', 'AGE_GR', 'STRMID', 'SSS']).agg(
    nd_fsu = ('UN_FSU', 'nunique'),
    ypstr=('YP', 'sum'),
    ymstr=('YM', 'sum'),
    yfstr=('YF', 'sum'),
    yhstr=('YH', 'sum')
).reset_index()

print(w2)

w2.to_excel('W2_TEST.xlsx')

       SEC  AGE_GR    STRMID SSS  nd_fsu     ypstr     ymstr     yfstr   yhstr
0        1       1  01010101   1       1     71.54     35.77     35.77     0.0
1        1       1  01010101   6       1    214.62     35.77    178.85     0.0
2        1       1  01010101   7       1   4793.40      0.00   4793.40     0.0
3        1       1  01010111   7       1  40277.25  40277.25      0.00     0.0
4        1       1  01010201   1       1     70.34     70.34      0.00     0.0
...    ...     ...       ...  ..     ...       ...       ...       ...     ...
96697    2       5  02363222   6       2   1490.00    894.00    596.00  1192.0
96698    2       5  02363223   2       1    269.00    269.00      0.00     0.0
96699    2       5  02363223   4       1    807.00    538.00    269.00     0.0
96700    2       5  02363223   6       1    538.00    269.00    269.00     0.0
96701    2       5  02363223   7       2  58238.50  34835.50  23403.00     0.0

[96702 rows x 9 columns]


In [30]:
# w2['nd_fsu']

In [31]:
# rteyuwid

In [32]:
# df_ndfsu

In [33]:




# Step 3: Generate STRMID (X SSS) X FSU level Estimates of each variable for the same combination
df['NS'] = df['NSS']
w3 = df.groupby(['SEC', 'AGE_GR', 'STRMID', 'SSS', 'FSU', 'NS']).agg(
    ypfsu=('YP', 'sum'),
    ymfsu=('YM', 'sum'),
    yffsu=('YF', 'sum'),
    yhfsu=('YH', 'sum')
).reset_index()

w3.to_excel('W3_TEST.xlsx')

# Step 4: Combine all three estimates at STRMID (X SSS) level for the same combination
w4 = pd.merge(w3, w2, on=['SEC', 'AGE_GR', 'STRMID', 'SSS'], suffixes=('', '_w2'))

w4.to_excel('W4_TEST.xlsx')

# Combining YHAT & XHAT
w1['r1hat'] = w1['yfhat'] / w1['ymhat']
w1['r2hat'] = w1['yhhat'] / w1['yphat']

w5 = pd.merge(w4, w1[['SEC', 'AGE_GR', 'yphat', 'ymhat', 'yfhat', 'yhhat', 'r1hat', 'r2hat']], on=['SEC', 'AGE_GR'])

w5.to_excel('W5_TEST.xlsx')


### Assuming you have a DataFrame 'wf_ppswr' with the necessary columns

# Step 5: Calculate variance of variables Y, X & R at STRMID level
def calculate_myfym(x):
    return np.sum(((x['NS'] * (x['yffsu'] - x['r1hat'] * x['ymfsu'])) - (x['yfstr'] - x['r1hat'] * x['ymstr'])) ** 2)

def calculate_vyh(x):
    return np.sum((x['NS'] * x['yhfsu'] - x['yhstr']) ** 2)

def calculate_myhyp(x):
    return np.sum(((x['NS'] * (x['yhfsu'] - x['r2hat'] * x['ypfsu'])) - (x['yhstr'] - x['r2hat'] * x['ypstr'])) ** 2)

w6 = w5.groupby(['SEC', 'AGE_GR', 'STRMID', 'SSS', 'NS', 'nd_fsu']).apply(
            lambda x: pd.Series({
                'myfym': calculate_myfym(x),
                'vyh': calculate_vyh(x),
                'myhyp': calculate_myhyp(x)
            })
        ).reset_index()

w6.to_excel('W6_TEST.xlsx')

w6A = w6.copy()
w6A.to_excel('W6A_TEST.xlsx')



####################################################################################################################

def calculate_myfym_w7(x):
    return np.sum(((x['NS'] - x['nd_fsu']) * (0 - (x['yfstr'] - x['r1hat'] * x['ymstr']))) ** 2)

def calculate_vyh_w7(x):
    return np.sum(((x['NS'] - x['nd_fsu']) * (0 - x['yhstr'])) ** 2)

def calculate_myhyp_w7(x):
    return np.sum(((x['NS'] - x['nd_fsu']) * (0 - (x['yhstr'] - x['r2hat'] * x['ypstr']))) ** 2)

w7 = w5.groupby(['SEC', 'AGE_GR', 'STRMID', 'SSS', 'NS', 'nd_fsu']).apply(
    lambda x: pd.Series({
        'myfym': calculate_myfym_w7(x),
        'vyh': calculate_vyh_w7(x),
        'myhyp': calculate_myhyp_w7(x)
    })
).reset_index()

w7.to_excel('W7_TEST.xlsx')


# # Define the condition for updating
# condition = (
#     (w6['sec'] + w6['age_gr'] + w6['strmid'] + w6['sss']) ==
#     (w7['sec'] + w7['age_gr'] + w7['strmid'] + w7['sss'])
# )

# # Update w6 based on the condition
# w6.loc['myfym'] = w6.loc[condition, 'myfym'] + w7.loc[condition, 'myfym']
# w6.loc['vyh'] = w6.loc[condition, 'vyh'] + w7.loc[condition, 'vyh']
# w6.loc['myhyp'] = w6.loc[condition, 'myhyp'] + w7.loc[condition, 'myhyp']

# # Divide by ns*(ns-1)
# w6['vyh'] /= (w6['NS'] * (w6['NS'] - 1))
# w6['myfym'] /= (w6['NS'] * (w6['NS'] - 1))
# w6['myhyp'] /= (w6['NS'] * (w6['NS'] - 1))

# # Generating Variance at Group level
# w8 = w6.groupby(['SEC', 'AGE_GR']).agg(
#     vyh=('vyh', 'sum'),
#     myfym=('myfym', 'sum'),
#     myhyp=('myhyp', 'sum')
# ).reset_index()

# # Post related estimates from W1
# w9 = pd.merge(w8, w1, on=['SEC', 'AGE_GR'], suffixes=('', '_w1'))

# t1 = w9.copy()
# t1['R1Hat'] = t1['yfhat'] / t1['ymhat']
# t1['RSE_YfYm'] = np.where(t1['yfhat'] > 0, 100 * np.sqrt(t1['myfym']) / t1['yfhat'], 0.00)
# t1['RSE_YH'] = np.where(t1['yhhhat'] > 0, 100 * np.sqrt(t1['vyh']) / t1['yhhhat'], 0.00)
# t1['R2Hat'] = t1['yhhhat'] / t1['yphat']
# t1['RSE_YhYp'] = np.where(t1['yhhhat'] > 0, 100 * np.sqrt(t1['myhyp']) / t1['yhhhat'], 0.00)
# t1

In [34]:


# def calculate_myfym(x):
#     return np.sum(((x['NS'] - x['nd_fsu']) * (0 - (x['yfstr'] - x['r1hat'] * x['ymstr']))) ** 2)

# def calculate_vyh(x):
#     return np.sum(((x['NS'] - x['nd_fsu']) * (0 - x['yhstr'])) ** 2)

# def calculate_myhyp(x):
#     return np.sum(((x['NS'] - x['nd_fsu']) * (0 - (x['yhstr'] - x['r2hat'] * x['ypstr']))) ** 2)

# w7 = w5.groupby(['SEC', 'AGE_GR', 'STRMID', 'SSS', 'NS', 'nd_fsu']).apply(
#     lambda x: pd.Series({
#         'myfym': calculate_myfym(x),
#         'vyh': calculate_vyh(x),
#         'myhyp': calculate_myhyp(x)
#     })
# ).reset_index()

# w7

In [35]:
# dqguioe

In [36]:
w7,w6

(       SEC  AGE_GR    STRMID SSS   NS  nd_fsu         myfym  vyh         myhyp
 0        1       1  01010101   1  2.0       1  5.059695e-01  0.0      0.000000
 1        1       1  01010101   6  2.0       1  2.067594e+04  0.0      0.000000
 2        1       1  01010101   7  2.0       1  2.297668e+07  0.0      0.000000
 3        1       1  01010111   7  2.0       1  1.558379e+09  0.0      0.000000
 4        1       1  01010201   1  2.0       1  4.752894e+03  0.0      0.000000
 ...    ...     ...       ...  ..  ...     ...           ...  ...           ...
 96697    2       5  02363222   6  2.0       2  0.000000e+00  0.0      0.000000
 96698    2       5  02363223   2  2.0       1  7.610526e+04  0.0   3784.576737
 96699    2       5  02363223   4  2.0       1  7.994397e+04  0.0  34061.190634
 96700    2       5  02363223   6  2.0       1  4.722196e+01  0.0  15138.306949
 96701    2       5  02363223   7  2.0       2  0.000000e+00  0.0      0.000000
 
 [96702 rows x 9 columns],
        SEC

In [39]:
### Adding both the variances
condition = (
    (w6['SEC'].astype(str) + w6['AGE_GR'].astype(str) + w6['STRMID'].astype(str) + w6['SSS'].astype(str)) ==
    (w7['SEC'].astype(str) + w7['AGE_GR'].astype(str) + w7['STRMID'].astype(str) + w7['SSS'].astype(str))
)

# Update 'w6' based on the condition
w6.loc[condition, 'myfym'] += w7.loc[condition, 'myfym']
w6.loc[condition, 'vyh'] += w7.loc[condition, 'vyh']
w6.loc[condition, 'myhyp'] += w7.loc[condition, 'myhyp']

w6

,SEC,AGE_GR,STRMID,SSS,NS,nd_fsu,myfym,vyh,myhyp
0,1,1,01010101,1,2.0,1,1.011939e+00,0.0,0.000000e+00
1,1,1,01010101,6,2.0,1,4.135188e+04,0.0,0.000000e+00
2,1,1,01010101,7,2.0,1,4.595337e+07,0.0,0.000000e+00
3,1,1,01010111,7,2.0,1,3.116757e+09,0.0,0.000000e+00
4,1,1,01010201,1,2.0,1,9.505787e+03,0.0,0.000000e+00
...,...,...,...,...,...,...,...,...,...
96697,2,5,02363222,6,2.0,2,1.867982e+05,0.0,9.289135e+03
96698,2,5,02363223,2,2.0,1,1.522105e+05,0.0,7.569153e+03
96699,2,5,02363223,4,2.0,1,1.598879e+05,0.0,6.812238e+04
96700,2,5,02363223,6,2.0,1,9.444392e+01,0.0,3.027661e+04


In [42]:
mask = w6['NS'] > 1

# Apply the calculation using vectorized operations
w6.loc[mask, 'vyh'] /= w6.loc[mask, 'NS'] * w6.loc[mask, 'NS'] - 1
w6.loc[mask, 'myfym'] /= w6.loc[mask, 'NS'] * w6.loc[mask, 'NS'] - 1
w6.loc[mask, 'myhyp'] /= w6.loc[mask, 'NS'] * w6.loc[mask, 'NS'] - 1


In [43]:
# w6 = pd.read_csv('W6_ShoumendraDATA.csv')
w6

,SEC,AGE_GR,STRMID,SSS,NS,nd_fsu,myfym,vyh,myhyp
0,1,1,01010101,1,2.0,1,3.373130e-01,0.0,0.000000e+00
1,1,1,01010101,6,2.0,1,1.378396e+04,0.0,0.000000e+00
2,1,1,01010101,7,2.0,1,1.531779e+07,0.0,0.000000e+00
3,1,1,01010111,7,2.0,1,1.038919e+09,0.0,0.000000e+00
4,1,1,01010201,1,2.0,1,3.168596e+03,0.0,0.000000e+00
...,...,...,...,...,...,...,...,...,...
96697,2,5,02363222,6,2.0,2,6.226606e+04,0.0,3.096378e+03
96698,2,5,02363223,2,2.0,1,5.073684e+04,0.0,2.523051e+03
96699,2,5,02363223,4,2.0,1,5.329598e+04,0.0,2.270746e+04
96700,2,5,02363223,6,2.0,1,3.148131e+01,0.0,1.009220e+04


In [44]:
# w6 = pd.read_csv('W6_ShoumendraDATA.csv')

# Generating Variance at Group level
w8 = w6.groupby(['SEC', 'AGE_GR']).agg(
    vyh=('vyh', 'sum'),
    myfym=('myfym', 'sum'),
    myhyp=('myhyp', 'sum')
).reset_index()

w8.to_excel('W8_TEST.xlsx')

In [45]:
# Post related estimates from W1
w9 = pd.merge(w8, w1, on=['SEC', 'AGE_GR'])

w9.to_excel('W9_TEST.xlsx')

In [46]:
T1 = w9.copy()
T1['R1Hat'] = 1000 * T1['r1hat']
T1['RSE_YfYm'] = np.where(T1['yfhat'] > 0, 100 * (T1['myfym'] ** 0.5) / T1['yfhat'], 0.00)
T1['RSE_YH'] = np.where(T1['yhhat'] > 0, 100 * (T1['vyh'] ** 0.5) / T1['yhhat'], 0.00)
T1['R2Hat'] = 100 * T1['r2hat']
T1['RSE_YhYp'] = np.where(T1['yhhat'] > 0, 100 * (T1['myhyp'] ** 0.5) / T1['yhhat'], 0.00)
T1

,SEC,AGE_GR,vyh,myfym,myhyp,no_sam,yphat,ymhat,yfhat,yhhat,r1hat,r2hat,R1Hat,RSE_YfYm,RSE_YH,R2Hat,RSE_YhYp
0,1,1,0.000000e+00,6.671300e+11,0.000000e+00,26930,5.829931e+07,2.944240e+07,2.885691e+07,0.00,0.980114,0.000000,980.114192,2.830449,0.000000,0.000000,0.000000
1,1,2,0.000000e+00,9.130905e+11,0.000000e+00,36049,8.047559e+07,4.250508e+07,3.797051e+07,0.00,0.893317,0.000000,893.316788,2.516579,0.000000,0.000000,0.000000
2,1,3,2.024714e+11,1.719333e+12,1.903833e+11,79739,1.731144e+08,9.499555e+07,7.811886e+07,16358976.65,0.822342,0.094498,822.342314,1.678511,2.750589,9.449806,2.667217
3,1,4,1.522155e+12,1.244939e+12,1.219207e+12,206314,4.300690e+08,2.162573e+08,2.138117e+08,81883483.19,0.988691,0.190396,988.691023,0.521846,1.506722,19.039614,1.348474
4,1,5,3.113687e+10,4.173130e+11,3.039388e+10,53557,7.431650e+07,3.800328e+07,3.631322e+07,2509864.76,0.955529,0.033773,955.528506,1.778960,7.030516,3.377265,6.946127
5,2,1,0.000000e+00,1.749592e+11,0.000000e+00,10099,2.155388e+07,1.164983e+07,9.904048e+06,0.00,0.850146,0.000000,850.145508,4.223337,0.000000,0.000000,0.000000
6,2,2,0.000000e+00,2.371244e+11,0.000000e+00,12899,2.819638e+07,1.541015e+07,1.278623e+07,0.00,0.829727,0.000000,829.727481,3.808428,0.000000,0.000000,0.000000
7,2,3,1.759382e+11,6.609058e+11,1.467374e+11,30421,6.589402e+07,3.642584e+07,2.946819e+07,10451790.30,0.808991,0.158615,808.991399,2.758775,4.013187,15.861515,3.665048
8,2,4,1.768255e+12,7.298018e+11,1.151309e+12,95739,2.088585e+08,1.076129e+08,1.012456e+08,93498401.48,0.940831,0.447664,940.831299,0.843774,1.422225,44.766380,1.147603
9,2,5,1.087542e+11,1.896430e+11,7.713888e+10,24822,3.207825e+07,1.583684e+07,1.624141e+07,7336130.81,1.025546,0.228695,1025.545813,2.681296,4.495272,22.869485,3.785904


In [47]:
T1['sec_desc'] = ''
T1['agegr_desc'] = ''

In [48]:
T1 = T1[['SEC', 'AGE_GR', 'sec_desc', 'no_sam', 'agegr_desc', 'R1Hat', 'RSE_YfYm', 'RSE_YH', 'R2Hat', 'RSE_YhYp']]
T1

,SEC,AGE_GR,sec_desc,no_sam,agegr_desc,R1Hat,RSE_YfYm,RSE_YH,R2Hat,RSE_YhYp
0,1,1,,26930,,980.114192,2.830449,0.000000,0.000000,0.000000
1,1,2,,36049,,893.316788,2.516579,0.000000,0.000000,0.000000
2,1,3,,79739,,822.342314,1.678511,2.750589,9.449806,2.667217
3,1,4,,206314,,988.691023,0.521846,1.506722,19.039614,1.348474
4,1,5,,53557,,955.528506,1.778960,7.030516,3.377265,6.946127
5,2,1,,10099,,850.145508,4.223337,0.000000,0.000000,0.000000
6,2,2,,12899,,829.727481,3.808428,0.000000,0.000000,0.000000
7,2,3,,30421,,808.991399,2.758775,4.013187,15.861515,3.665048
8,2,4,,95739,,940.831299,0.843774,1.422225,44.766380,1.147603
9,2,5,,24822,,1025.545813,2.681296,4.495272,22.869485,3.785904


In [49]:
T1.to_excel('T1_TEST.xlsx')

In [50]:
T1

,SEC,AGE_GR,sec_desc,no_sam,agegr_desc,R1Hat,RSE_YfYm,RSE_YH,R2Hat,RSE_YhYp
0,1,1,,26930,,980.114192,2.830449,0.000000,0.000000,0.000000
1,1,2,,36049,,893.316788,2.516579,0.000000,0.000000,0.000000
2,1,3,,79739,,822.342314,1.678511,2.750589,9.449806,2.667217
3,1,4,,206314,,988.691023,0.521846,1.506722,19.039614,1.348474
4,1,5,,53557,,955.528506,1.778960,7.030516,3.377265,6.946127
5,2,1,,10099,,850.145508,4.223337,0.000000,0.000000,0.000000
6,2,2,,12899,,829.727481,3.808428,0.000000,0.000000,0.000000
7,2,3,,30421,,808.991399,2.758775,4.013187,15.861515,3.665048
8,2,4,,95739,,940.831299,0.843774,1.422225,44.766380,1.147603
9,2,5,,24822,,1025.545813,2.681296,4.495272,22.869485,3.785904
